In [1]:
import http.client
import hashlib
import json
import urllib
import random
import time
from langconv import *
from googletrans import Translator
translator = Translator()

def dic2str(N_dic,token = '/'):
    lst = []
    for key, value in N_dic.items():
        lst.append(key)
    return token.join(lst)
def dic2lst(N_dic):
    lst = []
    for key, value in N_dic.items():
        lst.append(key)
    return lst
def zh2en(content,token = ' / '):
    
    if isinstance(content, str):
        return translator.translate(content, dest='en').text
    elif isinstance(content, dict):
        txt = dic2lst(content)
        lst_out = []
        for t in txt:
            lst_out.append(translator.translate(t, dest='en').text)  
        return lst_out
    else:
        print('data type not support!')
        
def en2zh(content):
    return translator.translate('cow', dest='zh-tw').text

def toSimplified(s):
    return Converter('zh-hans').convert(s)
def toTraditional(s):
    return Converter('zh-hant').convert(s)

# print(t1)
# print(t2)

# Split the text with tokenizer "\n"

In [4]:
import copy as cp
with open('./raw/三隻小豬.txt','r', encoding = 'utf8') as f:
    txt = f.read()

txt_split = txt.split('\n')
txt_tmp = []
for t in txt_split:
    if t != '':
        txt_tmp.append(t)
txt_split = txt_tmp
print(txt_split[:5])

['從前，有一隻胖胖的豬媽媽，她生了三隻小豬。最大的小豬：豬大哥很貪睡，很懶惰。一天到晚都在打瞌睡。第二個小豬：豬二哥很愛吃，他也很懶惰。幸好最小的小豬：豬小弟是個勤勞的好孩子。常常努力的工作。', '有一天，豬媽媽告訴他們說：「你們都長大了，應該自己蓋房子，自己住，自己種田，自己生活。我要你們自己照顧自己。」', '豬大哥噘起嘴，懶洋洋走出家門。「咿咿．．．．．好討厭哦！」豬大哥想一想說：「蓋一棟稻草屋吧，那最簡單了。」', '稻草屋很快的蓋好了，豬大哥好得意，他馬上去找兩個弟弟來，很驕傲的對他們說：「我的房子蓋好了，很漂亮吧～，你們也快一點蓋吧！」', '豬二哥看見說：「哇，果然很漂亮，大哥，你真了不起啊！」']


# Extract foreword by listening the first "「" sign

In [11]:
foreword = []
for i in range(len(txt_split)):
    if '「' not in txt_split[i]:
        foreword.append(txt_split[i])
    else:
        break
print('foreword:')
print(foreword)

foreword:
['從前，有一隻胖胖的豬媽媽，她生了三隻小豬。最大的小豬：豬大哥很貪睡，很懶惰。一天到晚都在打瞌睡。第二個小豬：豬二哥很愛吃，他也很懶惰。幸好最小的小豬：豬小弟是個勤勞的好孩子。常常努力的工作。']


# Load PER LOC TIM

In [21]:
import pickle
with open('PER_dic.pkl', 'rb') as handle:
    PER_dic = pickle.load(handle)
with open('space_dic.pkl', 'rb') as handle:

    LOC_dic = pickle.load(handle)
with open('time_dic.pkl', 'rb') as handle:
    TIM_dic = pickle.load(handle)
with open('dia_lst.pkl', 'rb') as handle:
    dia = pickle.load(handle)
with open('dia_hist.pkl', 'rb') as handle:
    dia_h = pickle.load(handle)

    
per = dic2lst(PER_dic)
loc = dic2lst(LOC_dic)
tim = dic2lst(TIM_dic)

print(per)
print(loc)
print(tim)
assert(txt.count('「') == len(dia))  #check all dialogues correspond to each characters

['猪大哥', '猪二哥', '猪小弟', '大野狼', '妈妈', '好孩子', '弟弟', '哥哥', '小弟', '大哥', '家', '小弟弟', '兄弟', '小猪', '野狼']
['稻草屋', '木屋', '红砖房子', '门闩上']
['从前', '天黑']


In [27]:
import copy as cp
with open('./raw/三隻小豬.txt','r', encoding = 'utf8') as f:
    txt = f.read()

txt_split = txt.split('\n')
txt_tmp = []
for t in txt_split:
    if t != '':
        txt_tmp.append(t)
txt_split = txt_tmp

dia_idx = 0
for li,line in enumerate(txt_split):
    if li<len(foreword):
        continue
    print(line,'\n')
    print('>extract result:')
    
    for i in range(line.count('「' )):
        print(dia[dia_idx]+'：'+dia_h[dia_idx])
        dia_idx+=1
    print('=======================')

有一天，豬媽媽告訴他們說：「你們都長大了，應該自己蓋房子，自己住，自己種田，自己生活。我要你們自己照顧自己。」 

>extract result:
妈妈：「你们都长大了，应该自己盖房子，自己住，自己种田，自己生活。我要你们自己照顾自己。」
豬大哥噘起嘴，懶洋洋走出家門。「咿咿．．．．．好討厭哦！」豬大哥想一想說：「蓋一棟稻草屋吧，那最簡單了。」 

>extract result:
猪大哥：「咿咿．．．．．好讨厌哦！」
大哥：「盖一栋稻草屋吧，那最简单了。」
稻草屋很快的蓋好了，豬大哥好得意，他馬上去找兩個弟弟來，很驕傲的對他們說：「我的房子蓋好了，很漂亮吧～，你們也快一點蓋吧！」 

>extract result:
弟弟：「我的房子盖好了，很漂亮吧～，你们也快一点盖吧！」
豬二哥看見說：「哇，果然很漂亮，大哥，你真了不起啊！」 

>extract result:
猪二哥：「哇，果然很漂亮，大哥，你真了不起啊！」
可是豬小弟並不以為然，他說：「哥，你不擔心稻草屋會被風吹倒嗎？」 

>extract result:
小弟：「哥，你不担心稻草屋会被风吹倒吗？」
豬二哥想了想，說：「稻草屋會被風吹倒，那我就用木頭來蓋好了，木屋較重，不怕風吹。」 

>extract result:
猪二哥：「稻草屋会被风吹倒，那我就用木头来盖好了，木屋较重，不怕风吹。」
豬二哥決定以後，「咚咚咚，咚咚咚！」他很認真的工作，一會兒敲敲釘子，一會兒鋸木頭。很快的，一棟木屋蓋好了。 

>extract result:
None：「咚咚咚，咚咚咚！」
豬二哥趕忙把哥哥和弟弟都請來，驕傲的說：「你們看，這麼漂亮的房子！而且釘得很牢固，不會被風吹倒，我真的好棒呀！」 

>extract result:
弟弟：「你们看，这么漂亮的房子！而且钉得很牢固，不会被风吹倒，我真的好棒呀！」
豬小弟說：「木屋雖然不會被風吹倒，可是用力打，木頭會被打斷，房子就垮了。」 

>extract result:
小弟：「木屋虽然不会被风吹倒，可是用力打，木头会被打断，房子就垮了。」
「討厭！」豬二哥罵小弟說：「你以為你最聰明？看你搬磚頭搬一整天了，房子還沒蓋出來。笨蛋～～」 

>extract result:
猪二哥：「讨厌！」
小弟：「你以为你最聪明？看你搬砖头搬一整天了，房子还没盖出来。笨蛋～～」
豬

In [2]:
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'C:\Program Files\stanford-corenlp-full-2018-10-05')

sentence = 'Once upon a time, there was a fat pig mother who gave birth to three piglets, The biggest pig: Big pig is very sleepy, very lazy, Sleeping all day long, The second pig: the pig brother is very fond of eating, he is also very lazy, Fortunately, the smallest pig: the pig brother is a hardworking good boy, Work hard often.'
print('Tokenize:', nlp.word_tokenize(sentence))
# print('Part of Speech:', nlp.pos_tag(sentence))
# print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
# print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Tokenize: ['Once', 'upon', 'a', 'time', ',', 'there', 'was', 'a', 'fat', 'pig', 'mother', 'who', 'gave', 'birth', 'to', 'three', 'piglets', ',', 'The', 'biggest', 'pig', ':', 'Big', 'pig', 'is', 'very', 'sleepy', ',', 'very', 'lazy', ',', 'Sleeping', 'all', 'day', 'long', ',', 'The', 'second', 'pig', ':', 'the', 'pig', 'brother', 'is', 'very', 'fond', 'of', 'eating', ',', 'he', 'is', 'also', 'very', 'lazy', ',', 'Fortunately', ',', 'the', 'smallest', 'pig', ':', 'the', 'pig', 'brother', 'is', 'a', 'hardworking', 'good', 'boy', ',', 'Work', 'hard', 'often', '.']
Constituency Parsing: (ROOT
  (S
    (S
      (SBAR (RB Once) (IN upon)
        (S
          (S
            (NP-TMP (DT a) (NN time))
            (, ,)
            (NP (EX there))
            (VP (VBD was)
              (NP
                (NP (DT a) (JJ fat) (NN pig) (NN mother))
                (SBAR
                  (WHNP (WP who))
                  (S
                    (VP (VBD gave)
                      (NP (NN birth))


In [18]:
# _*_coding:utf-8_*_

# Other human languages support, e.g. Chinese
sentence = '從前，有一隻胖胖的豬媽媽，她生了三隻小豬。最大的小豬：豬大哥很貪睡，很懶惰。一天到晚都在打瞌睡。第二個小豬：豬二哥很愛吃，他也很懶惰。幸好最小的小豬：豬小弟是個勤勞的好孩子。常常努力的工作。'

with StanfordCoreNLP(r'E:\stanford-corenlp-full-2018-10-05', lang='zh') as nlp:
    print(nlp.word_tokenize(sentence))
#     print(nlp.pos_tag(sentence))
#     print(nlp.ner(sentence))
    print(nlp.parse(sentence))
#     print(nlp.dependency_parse(sentence))

['從前', '，', '有', '一', '隻', '胖胖', '的', '豬媽媽', '，', '她', '生了', '三隻小豬', '。', '最大', '的', '小豬', '：', '豬大哥', '很', '貪睡', '，', '很', '懶惰', '。', '一天到晚', '都', '在', '打瞌睡', '。', '第二', '個', '小豬', '：', '豬二', '哥', '很', '愛吃', '，', '他', '也', '很', '懶惰', '。', '幸好', '最', '小', '的', '小豬', '：', '豬小弟', '是', '個勤勞', '的', '好', '孩子', '。', '常常', '努力', '的', '工作', '。']
(ROOT
  (IP
    (IP
      (ADVP (AD 從前))
      (PU ，)
      (VP (VE 有)
        (NP
          (QP (CD 一)
            (CLP (M 隻)))
          (CP
            (IP
              (VP (VA 胖胖)))
            (DEC 的))
          (NP (NN 豬媽媽)))))
    (PU ，)
    (IP
      (NP (PN 她))
      (VP
        (VSB (VV 生了) (VV 三隻小豬))))
    (PU 。)
    (IP
      (NP
        (DNP
          (ADJP (JJ 最大))
          (DEG 的))
        (NP (NN 小豬)))
      (PU ：)
      (IP
        (NP (NN 豬大哥))
        (VP
          (VP
            (ADVP (AD 很))
            (VP (VV 貪睡)))
          (PU ，)
          (VP
            (ADVP (AD 很))
            (VP (VV 懶惰))))))
    (PU 。)
    (IP
      (V

In [14]:
def rm_dialogues(s,token1 = '“',token2 = '”'):
    lst1 = [pos for pos, char in enumerate(s) if char == token1]
    lst2 = [pos for pos, char in enumerate(s) if char == token2]
    print(str(len(lst1))+' dialogues found')
    if(len(lst1) != len(lst1)):
        print('dialogue error: not equal “ ” Apostrophe count')
        return
    for i in range(len(lst1)):
        s = s[0: lst1[i]:] + '#'*(lst2[i]-lst1[i]+1)+ s[lst2[i] + 1::]
    return s,lst1,lst2

f = open('./raw/三隻小豬.txt','r', encoding = 'utf8')
txt = f.read()
txt_nd,lst1,lst2 = rm_dialogues(txt,"「","」")


27 dialogues found


In [15]:
import re
import zhon
from zhon.hanzi import punctuation


punc = """！？｡。＂＃#＄％＆＇（）＊＋－／：；，＜＝"＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】 〔〕〖〗〘〙〚〛〜〝〞〟〰–—''‛“”„‟…‧﹏"""
re_punc = "[{}]+".format(punc)
clean_txt = re.sub(re_punc, "#", txt_nd)
clean_txt = re.sub("\n", "", clean_txt)
clean_txt = clean_txt.split('#')
print(clean_txt)

['從前', '有一隻胖胖的豬媽媽', '她生了三隻小豬', '最大的小豬', '豬大哥很貪睡', '很懶惰', '一天到晚都在打瞌睡', '第二個小豬', '豬二哥很愛吃', '他也很懶惰', '幸好最小的小豬', '豬小弟是個勤勞的好孩子', '常常努力的工作', '有一天', '豬媽媽告訴他們說', '豬大哥噘起嘴', '懶洋洋走出家門', '豬大哥想一想說', '稻草屋很快的蓋好了', '豬大哥好得意', '他馬上去找兩個弟弟來', '很驕傲的對他們說', '', '豬二哥看見說', '可是豬小弟並不以為然', '他說', '豬二哥想了想', '說', '豬二哥決定以後', '他很認真的工作', '一會兒敲敲釘子', '一會兒鋸木頭', '很快的', '一棟木屋蓋好了', '豬二哥趕忙把哥哥和弟弟都請來', '驕傲的說', '豬小弟說', '', '豬二哥罵小弟說', '豬小弟不慌不忙說', '豬小弟不理會哥哥的嘲笑', '他搬好了磚塊', '又搬水泥', '他把水泥和好了', '開始堆砌磚塊', '一塊一塊將抹上水泥的磚頭堆砌起來', '豬小弟心理想', '你們不要笑我', '等我蓋好了', '你們就知道了', '我的房子比什麼都堅固', '野狼來了我也不怕呢', '豬小弟繼續加油工作', '他趕呀趕', '趕到天黑', '月亮掛在天上了', '他的紅磚房子才好不容易蓋好了', '豬小弟正想休息', '卻聽到了大野狼的吼叫聲', '豬小弟猛吸一口氣說', '這時兩個豬哥哥也聽到了野狼的吼叫聲', '怕得發抖', '他們怕野狼來', '所以整夜都不敢安心睡覺', '隔天', '豬媽媽要三隻小豬到田裡工作', '沒想到走到半路的時候', '他們被一隻可怕的大野狼發現了', '野狼一直跟蹤他們', '野狼決定先吃豬大哥', '豬大哥剛回到家', '就聽到大野狼的聲音', '野狼叫門叫得好兇', '豬大哥嚇得臉都白了', '他趕緊拿一把大鎖', '把門緊緊鎖了起來', '可是野狼哈哈大笑說', '他說完話', '用力一吹', '果然把稻草屋吹倒了', '豬大哥嚇得直跑到豬二哥家', '', '大哥慌慌張張跑進豬二哥的木屋子', '趕忙把大門閂上', '怕野狼闖進來', '可是野狼吼叫說', '兩隻小豬怕得縮著身子擠在一起', '頭上直冒冷汗', '因為野狼怕把門打